In [1]:
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from base64 import b64encode
import requests
from collections import Counter
from bs4 import BeautifulSoup
import re
from datetime import datetime
from urllib.parse import urlparse, parse_qs, urlencode
import undetected_chromedriver as uc

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-notifications")  # Disable notifications

# Initialize the WebDriver
driver = uc.Chrome()
try:
    # Navigate to Hibid.com
    driver.get("https://www.allsurplus.com")

    # Click on the signing button
    print('before signing in')
    time.sleep(10)
    try:
        # Wait for the button to be clickable
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )

        # Click the button
        accept_button.click()

        print("Cookies accepted successfully")
    except:
        print('cookies not set')   
    
    
    
    
    sign_in_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Sign In']"))
    )
    sign_in_button.click()
    print('after signing in')

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 3000);")

    login_form = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "login-standard"))
    )

    # Find and send keys to email input field
    email_input = login_form.find_element(By.NAME, "userName")
    email_input.send_keys("gsrecycle")

    # Wait for password field to be visible and interactable
    password_field =login_form.find_element(By.ID, "password")
    password_field.send_keys("Yekrut006")
    # Click the Sign In button
    sign_in_button = login_form.find_element(By.ID, "btnSignIn")
    sign_in_button.click()

    print('After clicking Sign In button')

    time.sleep(15)  # Increased wait time
    print("Login process completed.")
    watchlist_url = "https://www.allsurplus.com/account/watchlist"
    driver.get(watchlist_url)
    print('redirected to watch list')
    
    time.sleep(3)
    # Find all the product cards
    product_cards = driver.find_elements(By.CSS_SELECTOR, ".card.card-bids.mb-3.border-bottom")
    links=[]
    print(len(product_cards))
    for card in product_cards:
        link_element = card.find_element(By.TAG_NAME, 'a')
        product_link = link_element.get_attribute('href')
        print(product_link)
        links.append(product_link)
    # Extract data from each product card
    data_list = []
    for card in links:
        driver.get(card)
        time.sleep(3)
        try:
            time.sleep(3)
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h2.product-title.text-break").text
            except:
                title = "Title not found"
            time.sleep(3)
            # Scraping the location
            try:
                location = driver.find_element(By.CSS_SELECTOR, "span#lnkAssetDetailLocation").text
            except:
                location = "Location not found"
            time.sleep(3)
            try:
            # Scrape the container text
                container_text = driver.find_element(By.CSS_SELECTOR, "div.container-fluid:nth-child(4)").text
            # Use regular expression to find the full model number (including spaces between parts)
                model_match = re.search(r"Model\s*(.*?)(?=\n|$)", container_text)
                if model_match:
                    model_number = model_match.group(1).strip()
                else:
                    model_number = "Model not found"
            except:
                model_number = "Model not found"
            time.sleep(3)
            # Scraping the lot number using CSS Selector
            try:
                asset_detail_header = driver.find_element(By.CSS_SELECTOR, "#assetDetailHeader")
                asset_detail_text = asset_detail_header.text
            except:
                asset_detail_text = "Asset Detail Header not found"
            time.sleep(3)
            # Extract the lot number from the asset_detail_text
            try:
                # Locate the lot number, which follows "Lot#"
                lot_number = asset_detail_text.split('Lot#')[-1].strip()
            except:
                lot_number = "Lot number not found"
            time.sleep(3)
            # Scraping the current bid
            try:
                current_bid = driver.find_element(By.CSS_SELECTOR, "div#currentBid").text
            except:
                current_bid = "Current bid not found"
            time.sleep(3)
            # Scraping the closing date
            try:
                closing_date = driver.find_element(By.CSS_SELECTOR, "app-ux-timer p.timerAttribute span:last-of-type").text
            except:
                closing_date = "Closing date not found"

            # Append the data to the list
            data_list.append({
                "lot_number": lot_number,
                "title": title,
                "location": location,
                "model": model_number,
                "current_bid": current_bid,
                "start_date": "N.A", # Start date not available
                "closing_date": closing_date,
                "link": card
            })
        except Exception as e:
            print(f"Error processing link {card}: {e}")
except Exception as e:
    print(f"Error clicking button: {e}")
    # Create DataFrame
df1 = pd.DataFrame(data_list)
print(df1)
# Save data to CSV
filename = 'allsurplus.csv'
df1.to_csv(filename, index=False)
print(f"\nData saved to {filename}")
driver.quit()


before signing in
Cookies accepted successfully
after signing in
After clicking Sign In button
Login process completed.
redirected to watch list
7
https://www.allsurplus.com/asset/1046/9260
https://www.allsurplus.com/asset/1568/13215
https://www.allsurplus.com/asset/20528/13424
https://www.allsurplus.com/asset/296/24962
https://www.allsurplus.com/asset/662/19942
https://www.allsurplus.com/asset/628/19942
https://www.allsurplus.com/asset/661/19942
    lot_number                                              title  \
0    9260-1046                    DT5 Sokkisha Digital Theodolite   
1   13215-1568  Rohde & Schwarz CMW 500 WideBand Radio Communi...   
2  13424-20528                4pcs Waters Acquity UPLC Components   
3    24962-296                                Projector Screen 8'   
4    19942-662                          Audio and Video Equipment   
5    19942-628                             Lab and Test Equipment   
6    19942-661                             Lab and Test Equipment 

In [3]:
########################################################
########################################################
#################  EBAY CODE BELOW   ###################
########################################################
########################################################

# Initialize WebDriver (use your preferred driver, e.g., ChromeDriver)
driver = webdriver.Chrome(options=chrome_options)


def switch_tab_to_active(driver):
    # Get the current URL
    current_url = driver.current_url

    # Parse the URL
    parsed_url = urlparse(current_url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Check and modify the 'tabName' parameter
    if 'tabName' in query_params:
        current_tab = query_params['tabName'][0]
        if current_tab == 'SOLD':
            query_params['tabName'][0] = 'ACTIVE'
        elif current_tab == 'ACTIVE':
            query_params['tabName'][0] = 'SOLD'
        else:
            print(f"Tab name is not 'SOLD' or 'ACTIVE', it's: {current_tab}")
    else:
        # If 'tabName' is not present, set it to 'ACTIVE' by default
        query_params['tabName'] = ['ACTIVE']

    # Reconstruct the URL with modified parameters
    new_query = urlencode(query_params, doseq=True)
    new_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{new_query}"

    # Navigate to the new URL
    driver.get(new_url)
    print(f"Navigated to: {driver.current_url}")

def fetch_price_and_process(driver, average_listed_prices, average_sold_prices):
    switch_tab_to_active(driver)

    # Parse the URL
    current_url = driver.current_url
    parsed_url = urlparse(current_url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Check and modify the 'tabName' parameter
    if 'tabName' in query_params:
        current_tab = query_params['tabName'][0]
        if current_tab == 'SOLD':
            query_params['tabName'][0] = 'ACTIVE'  # Switch tab to 'ACTIVE' if 'SOLD' is present
            is_sold = True
        elif current_tab == 'ACTIVE':
            query_params['tabName'][0] = 'SOLD'  # Switch tab to 'SOLD' if 'ACTIVE' is present
            is_sold = False
        else:
            print(f"Tab name is not 'SOLD' or 'ACTIVE', it's: {current_tab}")
    else:
        # If 'tabName' is not present, set it to 'ACTIVE' by default
        query_params['tabName'] = ['ACTIVE']
        is_sold = False  # Default to 'ACTIVE'

    # Process page and extract prices
    time.sleep(5)
    current_url = driver.current_url
    try:
        try:
            # Wait for the tourtip to appear
            tourtip = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".tourtip__mask"))
            )
            
            # Find the close button within the tourtip
            close_button = tourtip.find_element(By.CSS_SELECTOR, ".tourtip__close")
            
            # Click on the close button
            close_button.click()
            
            print("Tourtip closed successfully.")
        except:
            pass
        time.sleep(3)
        html_content = driver.page_source
        
        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the .aggregates div
        aggregates = soup.find('div', class_='aggregates')
        
        if aggregates:
            # Find the first .metric-value element
            metric_value = aggregates.find('div', class_='metric-value')
            
            if metric_value:
                # Extract the text content
                avg_price = metric_value.text.strip()
                if is_sold:
                    average_sold_prices.append(avg_price)
                    print(f"Avg Sold Price: ${avg_price}")
                else:
                    average_listed_prices.append(avg_price)
                    print(f"Avg Listed Price: ${avg_price}")
            else:
                print("Metric value not found.")
                if is_sold:
                    average_sold_prices.append('0')
                else:
                    average_listed_prices.append('0')
        else:
            print("Aggregates div not found.")
            if is_sold:
                average_sold_prices.append('0')
            else:
                average_listed_prices.append('0')

    except Exception as e:
        print(f'An error occurred while trying to get the average listing price: {str(e)}')




# Navigate to eBay research page
url = "https://www.ebay.com/sh/research?marketplace=EBAY-US&tabName=SOLD"
driver.get(url)
print('waiting for 15 seconds')
time.sleep(60)
print('waiting done for 15 seconds')

# Wait for email field to be present
email_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "userid"))
)

# Fill in email address
email_field.send_keys("zainiawan5@gmail.com")

# Locate Continue button
continue_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "signin-continue-btn"))
)

# Click Continue button
continue_button.click()
time.sleep(30)
# Wait for password field to be present
password_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "pass"))
)

# Fill in password
password_field.send_keys("254ed228")

# Locate Sign in button
sign_in_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "sgnBt"))
)

# Click Sign in button
sign_in_button.click()

time.sleep(30)
try:
    try:
        # Wait for Skip for now button to be clickable
        skip_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "passkeys-cancel-btn"))
        )

        # Click Skip for now button
        skip_button.click()
        time.sleep(30)
    except:
        pass
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    # for index, row in df.iterrows():    
    #     try:
    #         search_input = WebDriverWait(driver, 10).until(
    #             EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
    #         )
    #         search_input.clear()
    #         search_input.send_keys(f"{row['title']}, {row['model']}")
    #         time.sleep(3)
    #         research_button = WebDriverWait(driver, 10).until(
    #             EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-input-panel__research-button"))
    #         )
    #         driver.execute_script("arguments[0].scrollIntoView(true);", research_button)
    #         research_button.click()            
            
    #     except:
    #         print('erro in the sending text try catch')
    average_listed_prices=[]
    average_sold_prices=[]


    for index, row in df1.iterrows():    
        try:
            # search_input = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
            # )
            # # Clear the existing text before sending new text
            # search_input.clear()
            # search_input.send_keys(f"{row['title']}, {row['model']}")
            
            search_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".search-input-panel__inputBox input"))
            )
            # Clear the existing text using JavaScript
            driver.execute_script("arguments[0].value = ''; arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", search_input)
            # Check if 'model' exists, is not empty, and does not contain "Model not found"
            if 'model' in row and row['model'].strip() and row['model'].strip().lower() != "model not found":
                
                search_input.send_keys(f"{row['model']} {row['title']}")
            else:
                # Search by title
                search_input.send_keys(f"{row['title']}")         
            
            
            time.sleep(3)
            research_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-input-panel__research-button"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", research_button)
            research_button.click()            
        except:
            print('error in the sending text try catch')

















        # Function to fetch and process the price


        for _ in range(2):
            fetch_price_and_process(driver, average_listed_prices, average_sold_prices)

        # You can check the result
        print("Avg Listed Prices:", average_listed_prices)
        print("Avg Sold Prices:", average_sold_prices)





















        switch_tab_to_active(driver)
        time.sleep(3)
except Exception as e:
    print(f"An error occurred: {e}")
# Close browser after processing
driver.quit()




df2_columns = pd.DataFrame({
    'average_listed_price': average_listed_prices,
    'average_sold_prices': average_sold_prices
})

# Concatenate only the last two columns to df
df = pd.concat([df1, df2_columns], axis=1)
filename = 'allsurplus.csv'
df.to_csv(filename, index=False)
print(f"\nData saved to {filename}")

waiting for 15 seconds
waiting done for 15 seconds
Navigated to: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=DT5+Sokkisha+Digital+Theodolite&dayRange=30&endDate=1739266350277&startDate=1736674350277&categoryId=0&tabName=ACTIVE&tz=Asia%2FKarachi
Avg Listed Price: $$277.00
Navigated to: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=DT5+Sokkisha+Digital+Theodolite&dayRange=30&endDate=1739266350277&startDate=1736674350277&categoryId=0&offset=0&limit=50&tabName=SOLD&tz=Asia%2FKarachi
Aggregates div not found.
Avg Listed Prices: ['$277.00']
Avg Sold Prices: ['0']
Navigated to: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=DT5+Sokkisha+Digital+Theodolite&dayRange=30&endDate=1739266350277&startDate=1736674350277&categoryId=0&tabName=ACTIVE&tz=Asia%2FKarachi
Navigated to: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=CMW+500+Rohde+%26+Schwarz+CMW+500+WideBand+Radio+Communication+Tester&dayRange=30&endDate=1739266438809&startDate=